In [ ]:
%load_ext autoreload
%autoreload 2

# HDRUK Acute Admissions Data Extraction

This is an example using sythetic data generated by LTHTR.

It is still under development and there are bound to be breaking changes and bugs. 

Please see commit history, ensure you have the latest clone of the repo, have updated your conda environment using the environment.yaml file.

Please raise an issue if you find a bug or have a question.

The pseduodata used in this script uses artificial data generated using MIMIC-III as per the method described by https://github.com/nhsx/SynthVAE but there are many other methods available described here: https://nhsx.github.io/AnalyticsUnit/synthetic.html

## Admitted Care

In [ ]:
# Import Packages

import numpy as np
import pandas as pd

import avoidable_admissions as aa

In [ ]:
# Import Data. Typically df is used as a moniker for a dataframe. The dytype argument can be droped if you want (it coerces all the data to strings)

df = pd.read_csv("synthetic_data/hdruk_acute_admissions_basic_synthetic.csv", dtype=str)

In [ ]:
# We have now extracted our data and we can see it is very messy and incomplete. The so called SNOMED codes are actually ICD9 codes so we need to tidy them up

df.head()

In [ ]:
# Look at the columns like this

df.columns

In [ ]:
# Simple ReMapping

column_remapper = {
    'ROW_ID'     :'rowid', 
     'SUBJECT_ID' :'patientid', 
     'ETHNICITY'  :'ethnos', 
     'ADMITTIME'  :'admitdate', 
     'ADMITTIME.1':'admittime',
       'DISCHTIME':'dischdatetime', # Not required for study but can be used to calc LOS 
          'GENDER':'gender', 
             'AGE':'activage', 
     'SNOMED_CODE':'icd9_code'
}

In [ ]:
# Then we just apply this to the data to start tidying it up

df.columns = df.columns.map(column_remapper)
df.head()

## First Validation

In [ ]:
# At this point we are going to try and run the validator over the data. It will not go well but it allows you to start playing with the validator

# First make a copy of the data
dfa = df.copy()

In [ ]:
# Then run the validator over the data

good, bad = aa.data.validate.validate_admitted_care_data(dfa)

In [ ]:
print(f"""
Total number of rows in input data   : {dfa.shape[0]}
Number of rows that passed validation: {good.shape[0]}
Number of rows that failed validation: {bad.shape[0]}
""")

In [ ]:
bad[["schema_context", "column", "check", "check_number", "failure_case", "index"]]

That didn't work and in this case we would need an entirely fresh extract as so much is missing. In the next example we will use a more complete dataset